# Повтор спелл-чекера из "A Complex Approach to Spellchecking and Autocorrection for Russian" [Dereza et. al 2016]

Финальный проект по НИСу "Не мой язык: автоматизированные подходы к изучению интерференции".  
Выполнен студент(-к-)ами группы БКЛ182 *Екатериной Гриневской, Романом Казаковым, Ксенией Петуховой, Вероникой Смилга*.

Импортируем необходимые модули.

In [17]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
import Levenshtein
from tqdm import tqdm
import numpy as np
import deeppavlov
from deeppavlov import configs
from deeppavlov import build_model
from gensim.models.word2vec import Word2Vec, LineSentence
from gensim.models import KeyedVectors
import os
import Levenshtein
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pickle
import kenlm

## Классификатор

### Данные для обучения классификатора

#### Словарь словарных употреблений

Преобразуем текст с полными парадигмами для лемм в датафрейм.

In [204]:
def make_df(file_path):
    with open(file_path, encoding='windows-1251') as f:
        text = f.read()
    text = text.replace('€', 'я')
    all_forms = []
    words = text.split('\n \n')
    for i in tqdm(words): 
        spl = i.split('\n')
        for n in range(0, len(spl)):
            listi = [spl[0].split(' | ')[0].replace('  ', ''), spl[n].split(' | ')[0].replace('  ', '')]
            all_forms.append(listi)
    df = pd.DataFrame(all_forms, columns=['lemma', 'forms']).drop_duplicates(keep='first', inplace=False).dropna()
    df.to_csv('lemmas-forms.csv')
    return df

### Русский Учебный Корпус

Преобразовываем Русский Учебный Корпус в датафрейм. Оставляем только предложения с орфографическими ошибками и опечатками, токенизируем, каждому токену присваиваем значение 0 (нет ошибки) или 1 (есть ошибка).

In [19]:
def df_rlc(path_rlc):
    df = pd.read_csv(path_rlc)
    df = df.dropna(subset=['tags'])
    list_space = list(zip(list(df.sentence_id), list(df.tags)))
    list_id = []
    for id_s, tag in list_space:
        if ('Hyphen' in tag) or ('Space' in tag):
            list_id.append(id_s)
            
    for i in list_id:
        df = df[df.sentence_id != i]
    
    df = df[df['tags'].str.match('.*(Ortho|Misspell).*')== True]
    list_df = list(zip(list(df.sentence_id), list(df.original_text), list(df.quote)))
    sent_id = []
    original_texts = []
    quotes = []

    df_new = pd.DataFrame()
    df_new['sentence_id'] = list_df[0]
    df_new['original_text'] = list_df[1]
    df_new['quote'] = list_df[2]
    
    list_sent_id = []
    list_tokens = []
    list_mistake = []
    list_token_id = []
    counter = 0
    for index, row in df.iterrows():
        text = re.sub(r'[^\w^\s\']', '', row['original_text'])
        text = text.lower()
        text = word_tokenize(text)
        for token in text:
            list_sent_id.append(row['sentence_id'])
            list_tokens.append(token)
            list_token_id.append(counter)
            counter += 1
            if token in row['quote'] or token.upper() in row['quote']:
                list_mistake.append(1)
            else:
                list_mistake.append(0)
    
    df_final = pd.DataFrame()
    df_final['sentence_id'] = list_sent_id
    df_final['token'] = list_tokens
    df_final['mistake'] = list_mistake
    return df_final

### Серебряный стандарт ГИКРЯ

Из файла с ограниченной выдачей генерального корпуса русского языка получаем датафрейм со столбцами token_id, sentence_id, token.

In [20]:
def list_of_dicts(splitted_doc):
    tokens_id = []
    sents_id = []
    list_of_words = []
    sents_count = 200000
    tokens_count = 200000
    for sent in splitted_doc:
        if (sent.startswith('TEXTID'))or(sent.startswith('\nTEXTID')):
            continue
        else:
            words = sent.split('\n')
            sents_count += 1
            for word in words:
                word_info = word.split('\t')
                if (len(word_info) > 3)and('#' not in word_info[3]):
                    word_dict = {}
                    token = word_info[2].replace(' ', '').lower()
                    tokens_count += 1
                    word_dict['token_id'] = tokens_count
                    word_dict['sentence_id'] = sents_count
                    word_dict['token'] = token.replace('ё', 'е')
                    list_of_words.append(word_dict)
    return list_of_words

Для токенов из ГОЛД проверяем, есть ли они в нашем словаре несловарных употреблений. Создаем датафрейм, каждому токену присваеваем 0, если слово есть в словаре (нет ошибки), или 1, если слова нет в словаре (есть ошибка). 

In [21]:
def get_gold_mist(lemmas_forms: pd.DataFrame) -> pd.DataFrame:
    lemmas_forms.dropna()
    with open('GOLD_1_2_release_demonstrative.txt', encoding = 'utf-8') as f:
        init = f.read()
    init_splitted = init.split('\n\n')
    list_of_words = list_of_dicts(init_splitted)
    df = pd.DataFrame(list_of_words, columns = ['token_id', 'sentence_id', 'token'])
    forms = lemmas_forms['forms'].to_list()
    forms = [str(x).replace(' ', '') for x in forms]
    print('Маркирую ошибочные слова...')
    for word_info in tqdm(list_of_words):
        if word_info['token'] in forms:
            word_info['mistake'] = 0
        else:
            word_info['mistake'] = 1
    df = pd.DataFrame(list_of_words, columns = ['token_id', 'sentence_id', 'token', 'mistake'])
    df.to_csv('gold_df.csv', index=False)
    return df

## Классификатор

Объединяем два датафрейма (РУК и GOLD), добавляем в получившийся датафрейм признаки: 1) длина токена; 2) длина контекста(сколько слов в предложении, не считая токен); 3) наличие более двух повторяющихся букв (бинарный признак).

In [22]:
def df_with_features(df, df2):
    new_df = pd.concat([df, df2])
    t_id = [i for i in range(len(new_df))]
    new_df['token_id'] = t_id
    lenght_of_token = []
    list_len = []
    dict_len = {}
    for index, row in new_df.iterrows():
        lenght_of_token.append(len(row['token']))
        if row['sentence_id'] not in dict_len.keys():
            dict_len[row['sentence_id']] = 0
        else:
            dict_len[row['sentence_id']] += 1

    for value in dict_len.values():
        for i in range(1, value + 2):
            list_len.append(value)
            
    list_repeat = []
    blacklist = []
    for index, row in new_df.iterrows():
        token = row['token']
        rep = 0
        black = 0
        if len(token) > 2:
            for i in range(3, len(token)):
                if (token[i] == token[i-1]) and (token[i] == token[i-2]):
                    rep = 1
        list_repeat.append(rep)
        if (re.search(r'[0-9]', token)) or (re.search(r'[^а-яё]', token)) or (len(token) == 1):
            black = 1
        blacklist.append(black)
    
    new_df['lenght_of_token'] = lenght_of_token
    new_df['amount_context'] = list_len
    new_df['repeated_letters'] = list_repeat
    new_df['blacklisted'] = blacklist
    return new_df

Функция *data_for_model()* собирает данные для обучения модели вектороноего представления слов из данных, предоставленные в рамках RuEval2016, данных, предоставленных А. Феногеновой (за что ей большое спасибо), и РУК. Возвращает список предложений из этих источников.

In [25]:
def data_for_model():
    df = pd.read_csv('annotated_texts_rlc.csv').dropna(subset=['original_text', 'corrected_text'])
    
    all_sents = []
    for row in df.itertuples():
        all_sents.append(row[3])
        all_sents.append(row[4])
        
    for i in range(1, 4):
        for fh in os.listdir(path=f"./blogsFinal/{i}"):
            with open(f"./blogsFinal/{i}/{fh}", encoding='utf-8') as f:
                sents = f.readlines()
                all_sents.extend(sents)
    
    with open('source_sents.txt', encoding='utf-8') as f:
        sents1 = f.readlines()
    all_sents.extend(sents1)
    with open('corrected_sents.txt', encoding='utf-8') as f:
        sents2 = f.readlines()
    all_sents.extend(sents2)
    all_sents = list(set(all_sents))
    
    print('всего предложений для обучения модели:', len(all_sents))
    
    return all_sents

Функция *data_to_file(sents)* преобразует список предложений в токенизированные строки и записывает их в файл *for_word2vec.txt*.

In [26]:
def data_to_file(sents):
    for s in tqdm(all_sents):
        words = [w.lower() for w in word_tokenize(s) if w.isalpha()]
        new_sent = ' '.join(words)
        with open('for_word2vec.txt', "a", encoding="utf-8") as fh:
            fh.write(new_sent + '\n')

Функция *def model_emb_make(data)* берёт на вход название файла с предложениями и обучает модель *Word2Vec*. Её лучше не запускать, потому что обучение проходит долго, а модель уже сохранена нами.

In [27]:
# Функция не используется, так как модель нами уже создана
def model_emb_make(data):
    model = Word2Vec(LineSentence(data), size=200, window=5, min_count=1, iter=20)
    word_vectors = model.wv
    word_vectors.save('vectors.kv')

Функция *model_emb_use(df_feat)* принимает на вход DataFrame токенов с их признакамит для обучения, описанными выше, и возвращает их с пятым признаком — вектором представления слова. Кроме того, скачивает модель из файлов *vectors.kv* и *vectors.kv.vectors.npy* (!!!).

In [28]:
def model_emb_use(df_feat):
    word_vectors = KeyedVectors.load('vectors.kv')
    
    vects = []
    for index, row in tqdm(df_feat.iterrows()):
        try:
            if row['mistake'] == 0:
                vects.append(word_vectors.get_vector(row['token']))
            else:
                lst_sim = word_vectors.most_similar(row['token'], topn=30)
                lst_sim_2 = []
                for s_token, s_score in lst_sim:
                    if word_vectors.similarity(row['token'], s_token) >= 0.4\
                                    and Levenshtein.ratio(row['token'], s_token) >= 0.3:
                        lst_sim_2.append(tuple([s_token, word_vectors.get_vector(s_token)]))
                if len(lst_sim_2) == 0:
                    vects.append(None)
                elif len(lst_sim_2) > 1:
                    res = np.stack(tuple([x[1] for x in lst_sim_2]))
                    vects.append(np.mean(res, axis = 0))
                else:
                    vects.append(lst_sim_2[0][1])
        except:
            vects.append(None)
            
    df_feat['vectors'] = vects
    df_feat = df_feat.replace(to_replace='None', value=np.nan).dropna()
    
    return df_feat

Функция *make_list(array)* принимает объект типа ndarray и преобразует его в list.

In [29]:
def make_list(array):
    return list(array)

Функция *preproc(df)* принимает на вход DataFrame с признаками для обучения и возвращает тренировочную и тестовую выборки для признаков и классов (0 — правильно, 1 — с ошибкой) в формате list.

In [214]:
def preproc(df):
    df['vectors'] = df['vectors'].apply(make_list)
    
    features = []
    results = []
    for row in tqdm(df.itertuples()):
        f = []
        f.append(row.lenght_of_token)
        f.append(row.amount_context)
        f.append(row.repeated_letters)
        f.append(row.blacklisted)
        f.extend(row.vectors)
        features.append(f)
        results.append(row.mistake)
    train_x, test_x, train_y, test_y = train_test_split(features, results, 
                                                        test_size=0.2, random_state=3)
    return train_x, test_x, train_y, test_y

Функция *model(train_x, test_x, train_y, test_y)* принимает на вход тренировочную и тестовую выборки для признаков и классов и обучает модель логистической регрессии. Кроме того, печатает отчёт об оценке качества классификатора.

In [31]:
def model(train_x, test_x, train_y, test_y):
    mod = LogisticRegression(random_state=0, max_iter = 200)
    mod.fit(train_x, train_y)
    target_names=['Правильные', 'Ошибки']
    report = metrics.classification_report(test_y, mod.predict(test_x),
                                  target_names=target_names)
    print(report)
    
    with open('final_model.pkl','wb') as f:
        pickle.dump(mod,f)

Функция *main()* собирает все вышеописанные функции, ничего не принимает и ничего не возвращает.

In [34]:
def main():
    making_df = make_df('Полная_парадигма_Морфология_Орфоэпия_Частотность.txt')
    rlc = df_rlc('annotated_texts_rlc.csv')
    gold_mist = get_gold_mist(making_df)
    features = df_with_features(rlc, gold_mist)
    
    # data_file_name = data_to_file(data_for_model())
    # model_emb_make(data_file_name)
    preprocessed = preproc(model_emb_use(features))
    model(preprocessed[0], preprocessed[1], preprocessed[2], preprocessed[3])

In [36]:
main()

  0%|          | 2/20885 [00:00<24:41, 14.10it/s]

Маркирую ошибочные слова...


100%|██████████| 20885/20885 [21:51<00:00, 15.92it/s]
/Users/romankazakov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
169406it [07:34, 372.45it/s]
/Users/romankazakov/.local/lib/python3.7/site-packages/pandas/core/missing.py:48: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
165989it [00:02, 71237.56it/s]
/Users/romankazakov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

  Правильные       0.94      0.99      0.96     29928
      Ошибки       0.79      0.42      0.55      3270

    accuracy                           0.93     33198
   macro avg       0.86      0.70      0.75     33198
weighted avg       0.92      0.93      0.92     33198



### Получаем варианты с пробелом

Функция, получающая все присутствующие в словаре варианты разделения слова на пробелы или дефисы. На вход получает строку (слово), на выходе - список строк с вариантами разделения.

In [207]:
def get_possible_split(word):
    cands = []
    cands1 = []
    for i in range(1, len(word)):
        words = []
        words.append(''.join(list(word)[:i]))
        words.append(''.join(list(word)[i:]))
        cands.append(words)
    for i in range(1, len(word)):
        words = []
        words.append(''.join(list(word)[:i]))
        words.append(''.join(list(word)[i:]))
        w = '-'.join(words)
        cands1.append(w)
    lemmas_word = pd.read_csv('lemmas-forms.csv').dropna()
    forms = lemmas_word['forms'].to_list()
    forms = [x.replace(' ', '') for x in forms]
    yes = []
    for cand in cands:
        if cand[0] in forms:
            if cand[1] in forms:
                yes.append(' '.join(cand))
    for cand in cands1:       
        if cand in forms:
            yes.append(cand)
    return yes

In [211]:
print(get_possible_split('коекак'))
print(get_possible_split('потомучто'))

['кое как', 'кое-как']
['потому что']


### Подбираем кандидатов с расстоянием Левенштейна 1, 2 и 3

Считаем расстояние Левенштейна слова до всех остальных слов из словаря и берём те, у которых оно равно 1, 2 или 3. На вход функция принимает строку, а на выходе даёт список словарей вида: {1: кандидаты, 2: кандидаты, 3: кандидаты}.

In [212]:
def get_candidates(string):
    lemmas_forms = pd.read_csv('lemmas-forms.csv')
    lemmas_forms.dropna()
    forms = lemmas_forms['forms'].to_list()
    forms = [str(x).replace(' ', '') for x in forms]
    candidates = {'1': [], '2': [], '3': []}
    print('Подбираю кандидатов...')
    for dict_word in tqdm(forms):
        try:
            lev_dist = Levenshtein.distance(string, dict_word)
            if lev_dist == 1:
                candidates['1'].append(dict_word)
            elif lev_dist == 2:
                candidates['2'].append(dict_word)
            elif lev_dist == 3:
                candidates['3'].append(dict_word)
            else:
                continue
        except:
            continue
    return candidates

In [213]:
print(get_candidates('малоко'))

  6%|▌         | 187002/3375170 [00:00<00:03, 934893.87it/s]

Подбираю кандидатов...


100%|██████████| 3375170/3375170 [00:02<00:00, 1287861.73it/s]


{'1': ['малок', 'малокто', 'малого', 'малого', 'малока', 'малоки', 'малоке', 'малоку', 'малокою', 'малокой', 'малок', 'малого', 'молоко'], '2': ['алого', 'балок', 'балок', 'валок', 'валок', 'валко', 'валко', 'валок', 'галок', 'далеко', 'далеко', 'жалок', 'жалко', 'жалко', 'локо', 'мавок', 'мазок', 'мазок', 'маклок', 'маклока', 'маклоку', 'маклоком', 'маклоке', 'маклоки', 'маклоков', 'маково', 'малой', 'малою', 'малакон', 'малек', 'малька', 'мальку', 'мальком', 'мальке', 'мальки', 'мальков', 'малик', 'малика', 'малику', 'маликом', 'малике', 'малики', 'маликов', 'малка', 'малки', 'малке', 'малку', 'малкою', 'малкой', 'мало', 'малоли', 'малочто', 'малое', 'малому', 'малом', 'малой', 'малому', 'малом', 'малою', 'малое', 'мало', 'малокам', 'малоках', 'малопе', 'малто', 'малому', 'малом', 'малой', 'малою', 'малое', 'мало', 'мамок', 'манок', 'манок', 'манко', 'манко', 'манок', 'марок', 'марок', 'марко', 'масок', 'матако', 'матико', 'маток', 'матово', 'матово', 'маток', 'мачок', 'мелок', 'мелк

## Строим модели

В силу того, что у нас не было возможности воссоздать корректор полностью (но части получилось (см. две функции выше)) из-за отсутствия подходящей языковой модели. Мы воспользовались подготовленными моделями от *deeppavlov* Левенштейна и Brill & Moore.

In [50]:
lev = build_model(deeppavlov.configs.spelling_correction.levenshtein_corrector_ru, download=True)
brill = build_model(deeppavlov.configs.spelling_correction.brillmoore_kartaslov_ru, download=True)

2020-12-23 20:00:52.545 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/vocabs/russian_words_vocab.dict.gz to /Users/romankazakov/.deeppavlov/downloads/russian_words_vocab.dict.gz
100%|██████████| 10.7M/10.7M [00:05<00:00, 1.87MB/s]
2020-12-23 20:00:58.336 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /Users/romankazakov/.deeppavlov/downloads/russian_words_vocab.dict.gz archive into /Users/romankazakov/.deeppavlov/downloads/vocabs
2020-12-23 20:00:58.620 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/lang_models/ru_wiyalen_no_punkt.arpa.binary.gz to /Users/romankazakov/.deeppavlov/downloads/ru_wiyalen_no_punkt.arpa.binary.gz
100%|██████████| 3.26G/3.26G [31:58<00:00, 1.70MB/s]   
2020-12-23 20:32:57.49 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /Users/romankazakov/.deeppavlov/downloads/ru_wiyalen_no_punkt.arpa.bi

## Интерфейс

Функция принимает на вход список токенов введенного предложения из функции *search()*, которая присваивает признаки для обучения для каждого из токенов и возвращает датафрейм.

In [125]:
def for_predict_feat(list_of_tokens):
    new_df = pd.DataFrame()
    new_df['token'] = list_of_tokens
    lenght_of_token = []
    for index, row in new_df.iterrows():
        lenght_of_token.append(len(row['token']))
    list_repeat = []
    blacklist = []
    
    word_vectors = KeyedVectors.load('vectors.kv')
    vects = []
    
    for index, row in new_df.iterrows():
        token = row['token']
        rep = 0
        black = 0
        if len(token) > 2:
            for i in range(3, len(token)):
                if (token[i] == token[i-1]) and (token[i] == token[i-2]):
                    rep = 1
        list_repeat.append(rep)
        if (re.search(r'[0-9]', token)) or (re.search(r'[^а-яё]', token)) or (len(token) == 1):
            black = 1
        blacklist.append(black)
        try:
            vects.append(word_vectors.get_vector(token))
        except KeyError:
            vects.append([None])
    
    new_df['lenght_of_token'] = lenght_of_token
    new_df['amount_context'] = [(len(list_of_tokens) - 1) for i in range(len(list_of_tokens))]
    new_df['repeated_letters'] = list_repeat
    new_df['blacklisted'] = blacklist
    new_df['vectors'] = vects
    new_df['vectors'] = new_df['vectors'].apply(make_list)
    return new_df

Принимает токен, если он есть в списке частотных ошибок русского языка, полученных с помощью ручной обработки, и датафрейм из функции *search()* и возвращает кортеж вида (токен, 1 (есть ошибка), исправление). 

In [126]:
def correct_popular(word, df):
    for i in range(len(df)):
        if df['original'].iloc[i] == word:
            correction = df['corrected'].iloc[i]
    return word, 1, correction

Преобразует список типа [float, float, float, float, list] в список, состоящий из первых четырех элементов и всех элементов из list.

In [179]:
def extend_list(list_df):
    new_list = list_df[0:4]
    new = list_df[4]
    new_list.extend(new)
    return new_list

Главная функция, предлагающая пользователю ввести предложение для исправления и выбрать модель, с помощью которой функция исправит опечатки. Возвращает датафрейм со столбцами: токен, наличие ошибки, исправление. 

In [215]:
def search():
    df_hype = pd.read_csv('popular_mistakes.tsv', sep='\t', header=0)
    sentence = input('Введите предложение: ')
    algo = int(input('Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore '))
    sentence =  re.sub(r'[^\w^\s\']', '', sentence)
    sentence = sentence.lower()
    sentence = word_tokenize(sentence)
    no_repeated = []
    df = for_predict_feat(sentence)
    corrections = []
    
    for token in sentence:
        if len(token) > 2:
            token = re.compile(r'(.)\1{2,}').sub(r'\1', token)   
        no_repeated.append(token)
     
    great_result = []
    with open('final_model.pkl', 'rb') as f:
        model = pickle.load(f)
    for token in no_repeated:
        if token in list(df_hype['original']):
            token, result, correction = correct_popular(token, df_hype)
        else:
            try:
                result = model.predict([extend_list(df.loc[df['token'] == token, 
                                                  ['lenght_of_token', 
                                                   'amount_context', 
                                                   'repeated_letters', 
                                                   'blacklisted', 
                                                   'vectors']].values.tolist()[0])])
            except:
                result = [1] 
            if result == [1]:
                if algo == 0:
                    correction = lev([token])[0]
                else:
                    correction = brill([token])[0]
            else: 
                correction = token
            
        great_result.append(int(result[0]))
        corrections.append(str(correction))
    
    
    quote = pd.DataFrame()
    quote['token'] = sentence
    quote['mistake'] = great_result
    quote['correction'] = corrections
    
    return quote

Инструкция для пользователя: введите предложение на русском языке, а затем введите номер модели. Запрустите функцию ниже.

In [ ]:
search()

### Примеры работы

In [216]:
search()

Введите предложение: Малако вкусное в моём халадилнике.
Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore 1


,token,mistake,correction
0,малако,1,молоко
1,вкусное,0,вкусное
2,в,0,в
3,моём,0,моём
4,халадилнике,1,холодильнике


In [217]:
search()

Введите предложение: Малако вкусное в моём халадилнике.
Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore 0


,token,mistake,correction
0,малако,1,малакон
1,вкусное,0,вкусное
2,в,0,в
3,моём,0,моём
4,халадилнике,1,халадилнике


Как мы видим, модель на основе расстояния Левенштейна работает плохо.

In [218]:
search()

Введите предложение: Призумпция невиновности.
Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore 1


,token,mistake,correction
0,призумпция,1,презумпция
1,невиновности,0,невиновности


In [219]:
search()

Введите предложение: Призумпция невиновности.
Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore 0


,token,mistake,correction
0,призумпция,1,презумпция
1,невиновности,0,невиновности


In [220]:
search()

Введите предложение: Ооооооооооччень крутой проэкт у наз.
Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore 1


,token,mistake,correction
0,ооооооооооччень,1,очень
1,крутой,0,крутой
2,проэкт,0,проэкт
3,у,0,у
4,наз,0,наз


In [221]:
search()

Введите предложение: Ооооооооооччень крутой проэкт у наз.
Выберите алгоритм: 0 - Левенштейн, 1 - Brill & Moore 0


,token,mistake,correction
0,ооооооооооччень,1,очень
1,крутой,0,крутой
2,проэкт,0,проэкт
3,у,0,у
4,наз,0,наз


Качество работы модели будет представлено на защите проекта.